In [ ]:
import os
import numpy as np
import pandas as pd
from joblib import load
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# 설정
CONFIG = {
    'backtesting_path': 'backtesting_raw_0702.xlsx',
    'model_dir': 'plk_folder_05_1',
    'output_path': 'predictions_v5_1.xlsx'
}

In [ ]:
def load_and_preprocess_data(file_path):
    """백테스팅 데이터를 로드하고 전처리합니다."""
    try:
        data = pd.read_excel(file_path, sheet_name=None)
        for keyword, df in data.items():
            df.reset_index(drop=True, inplace=True)
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
            df['date'] = df['date'].dt.strftime('%Y-%m-%d')
            df.set_index('date', inplace=True)
            print(f"{keyword} 키워드의 백테스트 데이터:")
            print(df.head())
        return data
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        return None

def load_models(model_dir, keywords):
    """저장된 모델을 로드합니다."""
    loaded_models = {}
    for keyword in keywords:
        model_path = os.path.join(model_dir, f"{keyword}_model.pkl")
        try:
            if os.path.exists(model_path):
                loaded_models[keyword] = load(model_path)
                print(f"{keyword} 키워드의 모델을 {model_path}에서 로드했습니다.")
            else:
                print(f"{keyword} 키워드의 모델 파일이 {model_path}에 존재하지 않습니다.")
        except Exception as e:
            print(f"{keyword} 키워드의 모델 로드 중 오류 발생: {e}")
    return loaded_models

In [ ]:
def predict_with_model(model, df):
    """모델을 사용하여 예측을 수행합니다."""
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    predictions = []
    for idx, row in enumerate(df_scaled):
        date = df.index[idx]
        X_test = row.reshape(1, -1)
        probabilities = model.predict_proba(X_test)[0]
        predictions.append({
            'date': date,
            'S_buy': probabilities[0],
            'buy': probabilities[1],
            'sell': probabilities[2]
        })
    return pd.DataFrame(predictions)

def save_predictions(predictions, output_path):
    """예측 결과를 엑셀 파일로 저장합니다."""
    try:
        with pd.ExcelWriter(output_path) as writer:
            for keyword, predictions_df in predictions.items():
                predictions_df.to_excel(writer, sheet_name=keyword, index=False)
        print(f"예측 결과가 {output_path}에 저장되었습니다.")
    except Exception as e:
        print(f"결과 저장 중 오류 발생: {e}")



In [ ]:
def main():
    # 데이터 로드 및 전처리
    backtesting_data = load_and_preprocess_data(CONFIG['backtesting_path'])
    if backtesting_data is None:
        return

    # 모델 로드
    loaded_models = load_models(CONFIG['model_dir'], backtesting_data.keys())

    # 예측 수행
    all_predictions = {}
    for keyword, df in tqdm(backtesting_data.items(), desc="예측 수행 중"):
        if keyword in loaded_models:
            model = loaded_models[keyword]
            predictions = predict_with_model(model, df)
            all_predictions[keyword] = predictions
            print(f"\n{keyword} 키워드의 예측 결과:")
            print(predictions.head())

    # 결과 저장
    save_predictions(all_predictions, CONFIG['output_path'])

if __name__ == "__main__":
    main()